In [1]:
import utils; reload(utils)
from utils import plots

Using cuDNN version 5103 on context None
Preallocating 10867/11439 Mb (0.950000) on cuda
Mapped name None to device cuda: Tesla K40c (0000:81:00.0)
Using Theano backend.


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [3]:
#path = "data/dogscats/sample/"
path = "data/state-farm/sample/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

# batch_size=100
batch_size=4

In [4]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [5]:
val_batches = get_batches(path+'valid', shuffle = False, batch_size=batch_size)
train_batches = get_batches(path+'train', shuffle = False, batch_size=batch_size)

Found 1000 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.


In [6]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [7]:
train_data = get_data(path+'train')
val_data = get_data(path+'valid')

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [8]:
save_array(model_path+'train_data.bc',train_data)
save_array(model_path+'val_data.bc',val_data)

In [9]:
train_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'val_data.bc')

In [10]:
#where do the things in this code come from?
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [11]:
train_classes = train_batches.classes
val_classes = val_batches.classes
train_labels = onehot(train_classes) #actual labels
val_labels = onehot(val_classes)

In [12]:
train_features = model.predict(train_data,batch_size=batch_size)
val_features = model.predict(val_data,batch_size=batch_size)

In [13]:
train_features.shape #will tell us input shape for the sequential linear model

(1500, 1000)

In [14]:
save_array(model_path+'train_lastlayer_features.bc',train_features)
save_array(model_path+'val_lastlayer_features.bc',val_features)

In [15]:
train_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'val_lastlayer_features.bc')

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [17]:
sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)

In [18]:
myModel = Sequential()

In [19]:
train_data.shape

(1500, 3, 224, 224)

In [20]:
myModel.add(BatchNormalization(axis=1, input_shape=(3,224,224)))
myModel.add(Convolution2D(32,3,3, activation='relu'))
myModel.add(BatchNormalization(axis=1))
myModel.add(MaxPooling2D(pool_size=(3,3)))
myModel.add(Convolution2D(64,3,3, activation='relu'))
myModel.add(BatchNormalization(axis=1))
myModel.add(MaxPooling2D(pool_size=(3,3)))
myModel.add(Flatten()) #flattens 3D shape to 2D for Dense layer
myModel.add(Dense(10,activation='softmax'))

In [21]:
myModel.compile(Adam(lr=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
#myModel.fit(train_features, train_labels, batch_size=batch_size, validation_data=(val_data,val_labels))
myModel.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 33s - loss: 14.3288 - acc: 0.1093 - val_loss: 14.5362 - val_acc: 0.0910
Epoch 2/2
1500/1500 [==============================] - 31s - loss: 14.3129 - acc: 0.1120 - val_loss: 14.6513 - val_acc: 0.0910


That didn't work so well (it feels as though the learning rate was too high?). Attempt 2:

In [23]:
model2 = Sequential()
model2.add(BatchNormalization(axis=1, input_shape=(3,224,224)))
model2.add(Convolution2D(32,3,3, activation='relu'))
model2.add(BatchNormalization(axis=1))
model2.add(MaxPooling2D(pool_size=(3,3)))
model2.add(Flatten()) #flattens 3D shape to 2D for Dense layer
model2.add(Dense(10,activation='softmax'))

In [24]:
#slow down learning rate by a factor of 10
myModel.compile(Adam(lr=1e-5),loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
myModel.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 14.3129 - acc: 0.1120 - val_loss: 14.6513 - val_acc: 0.0910
Epoch 2/2
1500/1500 [==============================] - 32s - loss: 14.3129 - acc: 0.1120 - val_loss: 14.6513 - val_acc: 0.0910


In [26]:
#try SGD?
myModel.compile(sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [27]:
myModel.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 14.3129 - acc: 0.1120 - val_loss: 14.6513 - val_acc: 0.0910
Epoch 2/2
1500/1500 [==============================] - 32s - loss: 14.3129 - acc: 0.1120 - val_loss: 14.6513 - val_acc: 0.0910


# Single Layer Attempt

In [29]:
simpleModel = Sequential()

In [36]:
simpleModel.add(BatchNormalization(axis=1, input_shape = (3,224,224)))
simpleModel.add(Convolution2D(data_format="channels_first",32,3,3, activation='relu'))
simpleModel.add(BatchNormalization(axis=1))
simpleModel.add(MaxPooling2D(pool_size=(3,3)))
simpleModel.add(Flatten())
simppleModel.add(Dense(10,activation='softmax'))

SyntaxError: non-keyword arg after keyword arg (<ipython-input-36-96a2a5ef9766>, line 2)

In [38]:
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.4116 - acc: 0.1220 - val_loss: 2.5205 - val_acc: 0.1160
Epoch 2/2
1500/1500 [==============================] - 32s - loss: 2.3394 - acc: 0.1127 - val_loss: 2.2715 - val_acc: 0.1700


# Simpler

In [41]:
modelA = Sequential()
modelA.add(BatchNormalization(axis=1, input_shape=(3,224,224)))
modelA.add(Flatten())
modelA.add(Dense(100, activation='relu'))
modelA.add(Dense(10,activation='softmax'))

In [45]:
modelA.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
modelA.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 31s - loss: 14.4700 - acc: 0.1000 - val_loss: 14.7319 - val_acc: 0.0860
Epoch 2/2
1500/1500 [==============================] - 31s - loss: 14.4633 - acc: 0.1027 - val_loss: 14.7319 - val_acc: 0.0860


In [44]:
#Jeremy's version
modelB = Sequential()
modelB.add(BatchNormalization(axis=1, input_shape=(3,224,224)))
modelB.add(Flatten())
modelB.add(Dense(100, activation='relu'))
modelB.add(BatchNormalization(axis=1))
modelB.add(Dense(10,activation='softmax'))

In [47]:
modelB.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
modelB.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 31s - loss: 2.3666 - acc: 0.1000 - val_loss: 2.3109 - val_acc: 0.1180
Epoch 2/2
1500/1500 [==============================] - 31s - loss: 2.3245 - acc: 0.1140 - val_loss: 2.3031 - val_acc: 0.1440


In [52]:
np.round(modelB.predict_generator(train_batches, train_batches.N)[:10],2)

AttributeError: 'DirectoryIterator' object has no attribute 'N'

# Model C

In [53]:
modelC = Sequential()
modelC.add(BatchNormalization(axis=1, input_shape=(3,224,224)))
modelC.add(Convolution2D(32,3,3,activation='relu'))
modelC.add(BatchNormalization(axis=1))
modelC.add(MaxPooling2D(pool_size=(3,3)))
modelC.add(Flatten())
modelC.add(Dense(100, activation='relu'))
modelC.add(BatchNormalization(axis=1))
modelC.add(Dense(10,activation='softmax'))

In [54]:
modelC.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
modelC.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 2.3736 - acc: 0.1033 - val_loss: 2.2936 - val_acc: 0.1130
Epoch 2/2
1500/1500 [==============================] - 32s - loss: 2.3244 - acc: 0.1053 - val_loss: 2.2665 - val_acc: 0.1770


In [56]:
modelC.optimizer.lr=0.001

In [57]:
modelC.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 2.3140 - acc: 0.1033 - val_loss: 2.2626 - val_acc: 0.1920
Epoch 2/2
1500/1500 [==============================] - 32s - loss: 2.3081 - acc: 0.1080 - val_loss: 2.2517 - val_acc: 0.1940


# Jeremy's ConvNet Function

In [58]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [59]:
conv1(train_batches)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 2.3946 - acc: 0.0913 - val_loss: 2.4171 - val_acc: 0.1250
Epoch 2/2
1500/1500 [==============================] - 32s - loss: 2.3378 - acc: 0.1093 - val_loss: 2.3056 - val_acc: 0.1260
Epoch 1/4
1500/1500 [==============================] - 32s - loss: 2.3207 - acc: 0.1160 - val_loss: 2.4294 - val_acc: 0.1180
Epoch 2/4
1500/1500 [==============================] - 32s - loss: 2.3128 - acc: 0.1247 - val_loss: 2.2608 - val_acc: 0.1800
Epoch 3/4
1500/1500 [==============================] - 32s - loss: 2.3072 - acc: 0.1180 - val_loss: 2.2462 - val_acc: 0.1790
Epoch 4/4
1500/1500 [==============================] - 32s - loss: 2.3024 - acc: 0.1187 - val_loss: 2.2387 - val_acc: 0.2160


# Attempt with Regularization

In [79]:
from keras import regularizers

regModel = Sequential()
regModel.add(BatchNormalization(axis=1,input_shape=(3,224,224), W_regularizer=l2(0.01)))
regModel.add(Convolution2D(32,3,3, activation='relu'))
regModel.add(BatchNormalization(axis=1))
regModel.add(MaxPooling2D(pool_size=(2,2)))
regModel.add(Convolution2D(64,3,3,activation='relu'))
regModel.add(BatchNormalization(axis=1))
regModel.add(MaxPooling2D(pool_size=(2,2)))
regModel.add(Flatten())
regModel.add(Dense(10,activation='softmax'))

TypeError: ('Keyword argument not understood:', 'W_regularizer')

In [68]:
regModel.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
regModel.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 35s - loss: 14.3015 - acc: 0.1113 - val_loss: 14.6191 - val_acc: 0.0930
Epoch 2/2
1500/1500 [==============================] - 32s - loss: 14.2914 - acc: 0.1133 - val_loss: 14.6191 - val_acc: 0.0930
